In [29]:
import pandas as pd
from gensim import utils
from gensim.models import FastText
from razdel import tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
from feature_extractor import FeatureExtractor
import math
import xgboost as xgb
import matplotlib.pyplot as plt
from autocorrect import Speller

In [30]:
ext = FeatureExtractor(
    town_index_path="/work/hack/additional_data/town_20230808.csv",
    district_index_path="/work/hack/additional_data/district_20230808.csv",
    street_abbv_index_path="/work/hack/additional_data/geonimtype_20230808.csv",
    town_abbv_index_path="/work/hack/additional_data/subrf_20230808.csv",
    area_type_index_path="/work/hack/additional_data/areatype_20230808.csv"
)

In [31]:
data = pd.read_csv("/work/hack/datasets/dataset_1.csv")

for i in range(1, 5):
    new_data = pd.read_csv(f"/work/hack/datasets/dataset_{i}.csv")
    data = pd.concat([data, new_data])

In [32]:
building = pd.read_csv("/work/hack/additional_data/building_20230808.csv")
valset = pd.read_csv("/work/hack/test_example.csv", sep=";")

<ipython-input-32-764e22617df3>:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv("/work/hack/additional_data/building_20230808.csv")


In [33]:
building = building[building["is_actual"] == True]

In [34]:
trainset = building["short_address"].tolist()
trainset = trainset + building["full_address"].tolist()
trainset = trainset + data[data["address"].notna()]["address"].tolist()
trainset = trainset + data[data["target_address"].notna()]["target_address"].tolist()

In [35]:
trainset = [ext.clear_text(i) for i in trainset]

In [36]:
# class MyIter:
#     def __iter__(self):
#         for index, row in building.iterrows():
#             tokens = list(tokenize(ext.resolve_abbv(row["full_address"])))
#             yield [_.text for _ in tokens]

class MyIter:
    def __iter__(self):
        for row in trainset:
            tokens = list(tokenize(row))
            yield [_.text for _ in tokens]      

model = FastText(vector_size=512, window=7, min_count=5, workers=30, sg=1, negative=25, seed=42)
model.build_vocab(corpus_iterable=MyIter())
total_examples = model.corpus_count
model.train(corpus_iterable=MyIter(), total_examples=total_examples, epochs=60)

(142735393, 296416920)

In [6]:
model = FastText.load("/work/hack/512_emb_fast_text.model")

In [37]:
vector_index = []

for index, row in tqdm(building.iterrows()):
    tokens = list(tokenize(ext.clear_text(row['full_address'])))
    tokens = [_.text for _ in tokens]
    predict = np.array([model.wv[token] for token in tokens])
    predict = np.mean(predict, axis=0)
    predict = predict / np.linalg.norm(predict)
    vector_index.append((row['id'], predict))

133462it [00:34, 3881.86it/s]


In [38]:
def get_id(text, target_id):
    index = [i[0] for i in vector_index].index(target_id)
    vector = model.wv[text] 
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    vector = np.array([model.wv[token] for token in tokens])
    vector = np.mean(vector, axis=0)
    vector = vector / np.linalg.norm(vector).reshape(1, -1) 
    max_sym = -1
    best_id = 0
    step = 10000

    all_syms = np.ones((1, 1))
    
    for ind in range(0, len(vector_index), step):
        vectors = np.array([i[1] for i in vector_index[ind: ind + step]]).T
        #vector_repeat = np.repeat(vector, vectors.shape[1], axis=0)
        #text_emb = np.hstack([vector_repeat, vectors.T])
        #xgb_syms = xgb_model.predict_proba(text_emb)[:, 1]
        syms = vector @ vectors
        #syms = syms * xgb_syms 
        all_syms = np.hstack([all_syms, syms])
        best_local_ind = np.argmax(syms)
        sym = syms[0][best_local_ind]
        if sym > max_sym:
            max_sym = sym
            best_id = vector_index[ind + best_local_ind][0]
    all_syms = all_syms[:, 1:]
    target_sym = all_syms[0][index]
    top_k = np.sum(np.where(all_syms > target_sym, True, False))
    
    return best_id, max_sym, top_k

In [39]:
cnt = 0
ind = 0
top_k_s = []
bads = []
for index, row in tqdm(valset.iterrows()):
    ind += 1
    if math.isnan(row['target_building_id']):
        continue
    best_id, max_sym, top_k = get_id(ext.clear_text(row['address']), row['target_building_id'])
    top_k_s.append(top_k)
    if top_k > 300:
        bads.append((best_id, max_sym, row['target_building_id'], index, top_k))
    if best_id == row['target_building_id']:
        cnt += 1
print(cnt / len(valset), sum(top_k_s) / len(top_k_s))
# 0.7103658536585366 988.1920731707318
# 0.716463414634146
# 0.6829268292682927 1012.4664634146342 для 20 эпох

0it [00:00, ?it/s]

328it [00:54,  5.98it/s]


0.6371951219512195 921.7073170731708


In [28]:
model.save("512_new_fasttext.model")

In [26]:
def get_top_n(text, n):
    vector = model.wv[text] 
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    vector = np.array([model.wv[token] for token in tokens])
    vector = np.mean(vector, axis=0)
    vector = vector / np.linalg.norm(vector).reshape(1, -1) 
    max_sym = -1
    best_id = 0
    step = 10000

    all_syms = np.ones((1, 1))
    
    for ind in range(0, len(vector_index), step):
        vectors = np.array([i[1] for i in vector_index[ind: ind + step]]).T
        syms = vector @ vectors
        all_syms = np.hstack([all_syms, syms])
        best_local_ind = np.argmax(syms)
        sym = syms[0][best_local_ind]
        if sym > max_sym:
            max_sym = sym
            best_id = vector_index[ind + best_local_ind][0]
    all_syms = all_syms[:, 1:]
    top_k = np.argpartition(all_syms[0], -n)[-n:].tolist()
    top_k = [(vector_index[i][0], all_syms[0][i]) for i in top_k]
    
    return best_id, max_sym, top_k

In [27]:
cnt = 0
ind = 0
top_k_s = []
bads = []
for index, row in tqdm(valset.iterrows()):
    ind += 1
    if math.isnan(row['target_building_id']):
        continue
    best_id, max_sym, top_k = get_top_n(ext.clear_text(row['address']), 300)
    top_k_inds = [i[0] for i in top_k]
    map_local_to_global = {i: top_k_inds[i] for i in range(len(top_k_inds))}
    map_global_to_local = {top_k_inds[i]: i for i in range(len(top_k_inds))}
    data_top_k_inds = building[building['id'].isin(top_k_inds)]
    features = ext.get_features(row['address'])
    for feature in features:
        if features[feature] is not None and \
        feature not in ["src_text", "preproc_text", "street", "structure", "country"]:
            data_top_k_inds = data_top_k_inds[(data_top_k_inds[feature] == features[feature]) | (data_top_k_inds[feature].isna())]
    if len(data_top_k_inds) != 0:
        data_top_k_inds['sim'] = data_top_k_inds['id'].apply(lambda x: top_k[map_global_to_local[x]][1])
        best_id = data_top_k_inds['id'].iloc[data_top_k_inds['sim'].argmax()]
    if best_id == row['target_building_id']:
        cnt += 1
print(cnt / len(valset))

279it [01:03,  5.68it/s]<ipython-input-27-825bc7ce4b25>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top_k_inds['sim'] = data_top_k_inds['id'].apply(lambda x: top_k[map_global_to_local[x]][1])
280it [01:03,  5.52it/s]<ipython-input-27-825bc7ce4b25>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top_k_inds['sim'] = data_top_k_inds['id'].apply(lambda x: top_k[map_global_to_local[x]][1])
281it [01:04,  4.35it/s]<ipython-input-27-825bc7ce4b25>:20: SettingWithCopyWarning: 
A value is trying to 

0.7469512195121951
